In [ ]:
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
# Lokasi file ZIP
zip_path = "/content/drive/MyDrive/data.zip"
extract_to = "/content"

# Ekstraksi file ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Dataset extracted to {extract_to}")

Dataset extracted to /content


#Menyiapkan dataset

In [ ]:
# Path ke dataset yang sudah dikelompokkan berdasarkan folder (sesuaikan path)
dataset_dir = "/content/data"

# Membuat ImageDataGenerator untuk latih, validasi, dan uji
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Memisahkan 20% data untuk validasi
)

# Dataset latih
train_dataset = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Ukuran gambar untuk MobileNetV2
    batch_size=32,
    class_mode='sparse',  # Kelas dalam format numerik
    subset='training'     # Data untuk pelatihan
)

# Dataset validasi
val_dataset = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',  # Kelas dalam format numerik
    subset='validation'   # Data untuk validasi
)

Found 2882 images belonging to 36 classes.
Found 720 images belonging to 36 classes.


#Membuat model

In [ ]:
# Load pre-trained MobileNetV2 tanpa top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Menonaktifkan training pada base model agar tidak terjadi overfitting
base_model.trainable = False

# Membangun model dengan menambahkan lapisan klasifikasi
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Mengubah output dari base model menjadi vektor
    Dense(1024, activation='relu'),  # Lapisan dense untuk klasifikasi
    Dense(train_dataset.num_classes, activation='softmax')  # Jumlah kelas untuk prediksi
])

# Menyusun model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 36)                  │          36,900 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,606,628 (13.76 MB)

 Trainable params: 1,348,644 (5.14 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

#Melatih model

In [ ]:
# Melatih model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


91/91 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.2234 - loss: 3.1295 - val_accuracy: 0.6167 - val_loss: 1.6735
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 214s 2s/step - accuracy: 0.6816 - loss: 1.4236 - val_accuracy: 0.7847 - val_loss: 1.0252
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.7989 - loss: 0.8763 - val_accuracy: 0.8014 - val_loss: 0.8227
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.8252 - loss: 0.6864 - val_accuracy: 0.8319 - val_loss: 0.6728
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 274s 2s/step - accuracy: 0.8521 - loss: 0.5729 - val_accuracy: 0.8389 - val_loss: 0.6119
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8810 - loss: 0.4761 - val_accuracy: 0.8514 - val_loss: 0.5464
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 267s 2s/step - accuracy: 0.8824 - loss: 0.4471 - val_accuracy: 0.8583 - val_loss: 0.5212
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.8815 - loss: 0.4284 - val_accuracy: 0.8403 - val_loss: 0.

#Evaluasi model

In [ ]:
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_acc:.2f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8744 - loss: 0.4621
Validation Accuracy: 0.88


In [ ]:
# Evaluasi pada dataset pelatihan
train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
print(f"Training Accuracy: {train_accuracy:.4f}, Training Loss: {train_loss:.4f}")

# Evaluasi pada dataset validasi
val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

# Analisis Performa
if val_accuracy < train_accuracy - 0.1:
    print("Model kemungkinan overfitting.")
elif train_accuracy - val_accuracy < 0.05:
    print("Model memiliki performa yang baik.")
else:
    print("Model mungkin underfitting.")

Training Accuracy: 0.9136, Training Loss: 0.3201
Validation Accuracy: 0.8639, Validation Loss: 0.4626
Model memiliki performa yang baik.


#finetuning

In [ ]:
# Melakukan fine-tuning pada beberapa layer
base_model.trainable = True
fine_tune_at = 100  # Mulai fine-tuning dari layer ke-100

# Freeze layer sebelum fine-tuning
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Menyusun ulang model untuk fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Melatih model lagi dengan fine-tuning
history_finetune = model.fit(
    train_dataset,
    epochs=10,  # Melanjutkan pelatihan
    validation_data=val_dataset
)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 348s 4s/step - accuracy: 0.7297 - loss: 0.9258 - val_accuracy: 0.6056 - val_loss: 1.4923
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 341s 3s/step - accuracy: 0.9016 - loss: 0.2913 - val_accuracy: 0.6181 - val_loss: 1.5080
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 290s 3s/step - accuracy: 0.9349 - loss: 0.1987 - val_accuracy: 0.7403 - val_loss: 1.0660
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 294s 3s/step - accuracy: 0.9484 - loss: 0.1699 - val_accuracy: 0.7667 - val_loss: 0.9099
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 293s 3s/step - accuracy: 0.9671 - loss: 0.1103 - val_accuracy: 0.7403 - val_loss: 1.0218
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 320s 3s/step - accuracy: 0.9582 - loss: 0.1258 - val_accuracy: 0.8000 - val_loss: 0.9081
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 292s 3s/step - accuracy: 0.9728 - loss: 0.0875 - val_accuracy: 0.7417 - val_loss: 1.1422
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 291s 3s/step - accuracy: 0.9711 - loss: 0.0813 - val_accuracy: 0.7972 - v

#Menyimpan model

In [ ]:
model.save("plant_disease_model.h5")

In [ ]:
#loaded_model = tf.keras.models.load_model("best_model.h5")

#prediksi

In [ ]:
# Ambil mapping kelas dari generator data latih
class_labels = train_dataset.class_indices  # train_dataset dari ImageDataGenerator
# Balikkan dictionary untuk memetakan indeks ke nama kelas
class_labels = {v: k for k, v in class_labels.items()}
print(class_labels)  # Output: {0: 'class_name1', 1: 'class_name2', ...}

{0: 'Apple___Apple_scab', 1: 'Apple___Black_rot', 2: 'Apple___Cedar_apple_rust', 3: 'Apple___healthy', 4: 'Blueberry___healthy', 5: 'Cherry___Powdery_mildew', 6: 'Cherry___healthy', 7: 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 8: 'Corn___Common_rust', 9: 'Corn___Northern_Leaf_Blight', 10: 'Corn___healthy', 11: 'Grape___Black_rot', 12: 'Grape___Esca_(Black_Measles)', 13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 14: 'Grape___healthy', 15: 'Orange___Haunglongbing_(Citrus_greening)', 16: 'Pepper,_bell___Bacterial_spot', 17: 'Pepper,_bell___healthy', 18: 'Potato___Early_blight', 19: 'Potato___Late_blight', 20: 'Potato___healthy', 21: 'Raspberry___healthy', 22: 'Soybean___healthy', 23: 'Squash___Powdery_mildew', 24: 'Strawberry___Leaf_scorch', 25: 'Strawberry___healthy', 26: 'Tomato___Bacterial_spot', 27: 'Tomato___Early_blight', 28: 'Tomato___Late_blight', 29: 'Tomato___Leaf_Mold', 30: 'Tomato___Septoria_leaf_spot', 31: 'Tomato___Spider_mites Two-spotted_spider_mite', 32: 'Tomato__

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Fungsi untuk memprediksi gambar
def predict_image(img_path):
    # Memuat dan memproses gambar
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Menambahkan batch dimension
    img_array = img_array / 255.0  # Normalisasi gambar

    # Prediksi dengan model
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=-1)

    # Ambil nama kelas berdasarkan prediksi
    class_names = list(train_dataset.class_indices.keys())
    predicted_class_name = class_names[predicted_class[0]]

    return predicted_class_name

# Contoh pemakaian
image_path = '/content/data/Apple___Black_rot/image (10).JPG'
predicted_class = predict_image(image_path)
print(f"Predicted Class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: Apple___Black_rot
